In [1]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from transformers import pipeline


In [2]:
# adapters_name  = "experiments/Llama-2-7b-hf/checkpoint-16000"
model_name = "/home/abdelrahman.sadallah/.cache/huggingface/hub/jais"



model = AutoModelForCausalLM.from_pretrained(model_name, device_map = 'auto' ,trust_remote_code=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
def gen(model, text: str, tmp=0.1, top_p=0.6, max_length=1024):
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=max_length,
                                do_sample=True,
                                top_p=top_p,
                                temperature=tmp)

    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [12]:
model

JAISLMHeadModel(
  (transformer): JAISModel(
    (wte): Embedding(84992, 5120)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-39): 40 x JAISBlock(
        (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (attn): JAISAttention(
          (c_attn): Linear8bitLt(in_features=5120, out_features=15360, bias=True)
          (c_proj): Linear8bitLt(in_features=5120, out_features=5120, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (mlp): JAISMLP(
          (c_fc): Linear8bitLt(in_features=5120, out_features=13653, bias=True)
          (c_fc2): Linear8bitLt(in_features=5120, out_features=13653, bias=True)
          (c_proj): Linear8bitLt(in_features=13653, out_features=5120, bias=True)
          (act): SwiGLUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
     

In [14]:
base_text = '''### Instruction: Below is a prompt to generate a poem. Generate a poem.


### Input:
'''

base_text = ''''''

# Happy kid was playing at the park, but then he broke his leg, and his life got completely changed.

with open('arabic_output.txt', 'a') as f:
    while 1:
        print('-'*20)
        text = input("Enter a prompt: ")
        print('-'*20)

        f.write('='*30 + '\n')
        f.write(text + '\n')
        f.write('-'*20 + '\n')

        text = base_text + text

        output = gen(model, text)

        output = output.replace('<newline>', '\n')

        f.write(output + '\n')
        f.write('='*30 + '\n')
        f.flush()

        print('-'*20)
        print(output)
        print('-'*20)


--------------------
--------------------


KeyboardInterrupt: 